In [1]:
from json import loads
from pyspark.sql import SparkSession
import warnings
warnings.filterwarnings("ignore")
from pyspark.sql.functions import col,from_json,udf
from pyspark.sql.types import StructType,StructField, StringType, IntegerType,MapType,FloatType,ArrayType,BooleanType

In [2]:
spark = SparkSession.\
        builder.\
        appName("process-data").\
        master("spark://spark-master:7077").\
        config("spark.executor.memory", "1024m").\
        getOrCreate()

22/11/21 13:55:52 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
Using Spark's default log4j profile: org/apache/spark/log4j-defaults.properties
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).


In [3]:
df_product = spark.read.parquet('hdfs://namenode:9000/TikiCleaned/Product')
df_comment = spark.read.parquet('hdfs://namenode:9000/TikiCleaned/Comment')
df_shop = spark.read.parquet('hdfs://namenode:9000/TikiCleaned/ShopInfo')
df_product_shop = spark.read.parquet('hdfs://namenode:9000/TikiCleaned/Product_Shop')

In [4]:
df_product.createOrReplaceTempView("product")
df_comment.createOrReplaceTempView("comment")
df_shop.createOrReplaceTempView("shop")
df_product_shop.createOrReplaceTempView("product_shop")

In [5]:
print("comment count: {} \nproduct count: {} \nshop count: {} \
      \nproduct_shop count: {}".format(df_comment.count(),df_product.count(),
                                       df_shop.count(),df_product_shop.count()))

comment count: 4243515 
product count: 46721 
shop count: 9945       
product_shop count: 127069


In [21]:
df_all = spark.sql("""
    select p.id p_id, p.master_id,p.price,p.list_price,p.original_price,p.discount,p.discount_rate,
    p.productset_group_name,p.all_time_quantity_sold,p.name p_name,p.description,p.specifications,
    p.category_name,p.category_id,
    ps.sellers, s.store_id,s.name s_name,s.icon,s.url,s.is_official,s.store_level,s.is_followed,
    s.avg_rating_point,s.review_count, s.total_follower,s.days_since_joined, 
    c.id c_id,c.contentRaw, c.content,c.thank_count,c.customer_id,c.rating,
    c.customer_full_name,c.purchased_at,c.date_purchased_at,c.review_created_date,
    c.delivery_date,c.review_after_delivery,c.sentiment,c.lable
    from product p
    join product_shop ps on p.id = ps.id
    join shop s on ps.sellers = s.id
    join comment c on c.product_id = p.id and ps.sellers = c.seller_id
""")

In [22]:
df_all.write.partitionBy("category_id").mode('append').parquet('hdfs://namenode:9000/TikiCleaned/DataAll')

In [23]:
df_all.count()

3800075